In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from PIL import Image
import numpy as np
from colorthief import ColorThief
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# Load the CSV data (replace with your actual file path)
csv_file = 'price_tracking/resource_prices_2024_10.csv'  # Update this path to the location of your CSV
df = pd.read_csv(csv_file)

# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set the Seaborn style for better aesthetics
sns.set(style='darkgrid')

# Set the font for all plots
rcParams['font.family'] = 'Consolas'
rcParams['font.size'] = 20

# Function to get dominant color from an image
def get_dominant_color(image_path):
    color_thief = ColorThief(image_path)
    dominant_color = color_thief.get_color(quality=1)
    return dominant_color

# Function to add image near the plot on the left side with transparency handling
def add_image_left_side(fig, ax, image_path):
    img = Image.open(image_path).convert('RGBA')  # Maintain transparency
    img = np.array(img)
    
    # Get the height of the axes and the figure in inches
    ax_bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    ax_height_in_inches = ax_bbox.height  # Height of the plot area
    
    # Calculate the zoom factor based on the plot height
    img_height_in_inches = img.shape[0] / fig.dpi  # Image height in inches
    zoom_factor = ax_height_in_inches / img_height_in_inches  # Ensure image fits within plot height
    zoom_factor = zoom_factor / 1.3  # Ensure image fits within plot height

    # Create the inset box with the image
    imagebox = OffsetImage(img, zoom=zoom_factor)
    ab = AnnotationBbox(imagebox, (-0.15, 0.5), frameon=False, xycoords='axes fraction', box_alignment=(1, 0.5))
    
    # Add the image to the current plot (ax)
    ax.add_artist(ab)

# Create a function to plot each resource separately
def plot_each_resource_separately():
    unique_resources = df['Resource'].unique()
    
    # Loop through each resource and plot its price separately
    for resource in unique_resources:
        fig, ax = plt.subplots(figsize=(15, 5))  # Increased height for better alignment with image
        
        # Filter data for the current resource
        resource_data = df[df['Resource'] == resource]
        
        # Path to the corresponding image
        image_path = f'assets/{resource}.png'  # Ensure image names match resource names
        
        # Get the dominant color from the image
        dominant_color = get_dominant_color(image_path)
        dominant_color_normalized = [x / 255.0 for x in dominant_color]  # Normalize for matplotlib
        
        # Plot the line using the dominant color
        sns.lineplot(data=resource_data, x='Date', y='P2P (SFL)', marker='x', linewidth=4, color=dominant_color_normalized, ax=ax)
        
        # Show the grid explicitly
        ax.grid(True)
        
        # Hide x and y labels but keep the grid
        ax.xaxis.label.set_visible(False)
        ax.yaxis.label.set_visible(False)
        
        # Add the image on the left side
        add_image_left_side(fig, ax, image_path)
        
        # Customize the appearance
        plt.tight_layout()
        
        # Show the plot for this resource
        plt.show()

# Call the function to plot each resource on a separate plot
plot_each_resource_separately()
